discuss if false negative or false positive is worse. here its worse to lose a customer when you think you have them, then to think ull lose one when u rly wont. So false negative, bc here if churn is true it means u lost the customer, so false negative means u predict no churn when they really do. Thus we should maximize recall to make sure we get all the cases where churn is true.

In [225]:
import pandas as pd
import numpy as np
np.random.seed(42)
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import precision_score, recall_score,\
classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV,\
cross_val_score, RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier

In [120]:
!ls data

archive.zip                        bigml_59c28831336c6604c800002a.csv


In [121]:
df = pd.read_csv('data/bigml_59c28831336c6604c800002a.csv')
df.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   state                   3333 non-null   object 
 1   account length          3333 non-null   int64  
 2   area code               3333 non-null   int64  
 3   phone number            3333 non-null   object 
 4   international plan      3333 non-null   object 
 5   voice mail plan         3333 non-null   object 
 6   number vmail messages   3333 non-null   int64  
 7   total day minutes       3333 non-null   float64
 8   total day calls         3333 non-null   int64  
 9   total day charge        3333 non-null   float64
 10  total eve minutes       3333 non-null   float64
 11  total eve calls         3333 non-null   int64  
 12  total eve charge        3333 non-null   float64
 13  total night minutes     3333 non-null   float64
 14  total night calls       3333 non-null   

In [123]:
df['area code'] = df['area code'].astype(object)

df.dtypes

state                      object
account length              int64
area code                  object
phone number               object
international plan         object
voice mail plan            object
number vmail messages       int64
total day minutes         float64
total day calls             int64
total day charge          float64
total eve minutes         float64
total eve calls             int64
total eve charge          float64
total night minutes       float64
total night calls           int64
total night charge        float64
total intl minutes        float64
total intl calls            int64
total intl charge         float64
customer service calls      int64
churn                        bool
dtype: object

In [227]:
df.churn.value_counts()

False    2850
True      483
Name: churn, dtype: int64

In [124]:
y = df.churn
X = df.drop(['churn', 'phone number'], axis = 1)

In [125]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [126]:
X_train_cat = X_train.select_dtypes('object')

ohe = OneHotEncoder(drop='first', sparse=False)

dums = ohe.fit_transform(X_train_cat)
dums_df = pd.DataFrame(dums, columns=ohe.get_feature_names(), index=X_train_cat.index)

In [127]:
dums_df.head()

,x0_AL,x0_AR,x0_AZ,x0_CA,x0_CO,x0_CT,x0_DC,x0_DE,x0_FL,x0_GA,...,x0_VA,x0_VT,x0_WA,x0_WI,x0_WV,x0_WY,x1_415,x1_510,x2_yes,x3_yes
817,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
56,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [128]:
X_train_nums = X_train.select_dtypes(['int64', 'float64'])

ss = StandardScaler()

ss.fit(X_train_nums)

nums_df = pd.DataFrame(ss.transform(X_train_nums), columns=X_train_nums.columns, index=X_train_nums.index)

In [129]:
nums_df.head()

,account length,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls
817,3.601382,-0.584936,-1.547653,-0.429657,-1.547170,-0.729987,-1.840891,-0.731087,1.255804,0.925634,1.256197,-1.300791,0.634849,-1.304132,0.318978
1373,0.184951,-0.584936,-1.244014,0.224176,-1.244071,-0.138082,0.499864,-0.139179,0.165090,-0.353704,0.164841,-2.194793,-0.184370,-2.191525,1.813519
679,-0.650176,-0.584936,0.787609,-1.133785,0.787772,2.491952,0.549667,2.493068,0.147339,0.209205,0.147309,-0.549828,1.863677,-0.549186,-0.428293
56,1.020079,-0.584936,-0.969818,-0.127888,-0.970200,-0.408385,-1.890695,-0.408439,-1.178086,1.437368,-1.176344,-0.800149,-1.003589,-0.800835,-0.428293
1993,-0.371801,-0.584936,0.675354,-0.228477,0.675192,1.294330,-1.143645,1.295326,0.265680,0.516246,0.265649,-2.051753,-0.593980,-2.045833,-1.175564


In [130]:
X_train_clean = pd.concat([nums_df, dums_df], axis=1)

X_train_clean.head()

,account length,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,...,x0_VA,x0_VT,x0_WA,x0_WI,x0_WV,x0_WY,x1_415,x1_510,x2_yes,x3_yes
817,3.601382,-0.584936,-1.547653,-0.429657,-1.547170,-0.729987,-1.840891,-0.731087,1.255804,0.925634,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1373,0.184951,-0.584936,-1.244014,0.224176,-1.244071,-0.138082,0.499864,-0.139179,0.165090,-0.353704,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
679,-0.650176,-0.584936,0.787609,-1.133785,0.787772,2.491952,0.549667,2.493068,0.147339,0.209205,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
56,1.020079,-0.584936,-0.969818,-0.127888,-0.970200,-0.408385,-1.890695,-0.408439,-1.178086,1.437368,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1993,-0.371801,-0.584936,0.675354,-0.228477,0.675192,1.294330,-1.143645,1.295326,0.265680,0.516246,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [131]:
X_test_cat = X_test.select_dtypes('object')

test_dums = ohe.transform(X_test_cat)

test_dums_df = pd.DataFrame(test_dums, columns=ohe.get_feature_names(), index=X_test_cat.index)

In [132]:
X_test_nums = X_test.select_dtypes(['int64','float64'])

test_nums = ss.transform(X_test_nums)

test_nums_df = pd.DataFrame(test_nums, columns=X_test_nums.columns, index=X_test_nums.index)

In [133]:
X_test_clean = pd.concat([test_nums_df, test_dums_df], axis=1)

X_test_clean.head()

,account length,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,...,x0_VA,x0_VT,x0_WA,x0_WI,x0_WV,x0_WY,x1_415,x1_510,x2_yes,x3_yes
438,0.311486,-0.584936,-0.452712,-0.379362,-0.452767,2.562980,0.300651,2.562705,-0.219520,1.181501,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2674,-0.852632,-0.584936,-1.297381,0.827714,-1.297113,0.329524,1.197110,0.329704,-0.239243,2.102624,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1345,-0.068118,-0.584936,-3.305080,-5.056782,-3.305141,-0.810881,1.495930,-0.810008,-0.659356,-0.609571,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1957,1.171920,-0.584936,0.610946,-1.083490,0.611325,0.067112,-0.446399,0.067408,-0.874343,0.669766,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2148,-0.118732,-0.584936,-0.655138,0.073292,-0.655194,0.473554,-1.342858,0.473619,0.535893,-0.456051,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### Logistic Regression

In [174]:
lr = LogisticRegression(random_state=42)

lr.fit(X_train_clean, y_train)

LogisticRegression(random_state=42)

In [175]:
scores = cross_val_score(estimator=lr, X = X_train_clean, y= y_train, cv=10)
scores

array([0.86142322, 0.86891386, 0.87265918, 0.86516854, 0.86142322,
       0.86142322, 0.84210526, 0.84962406, 0.84586466, 0.87593985])

In [176]:
np.median(scores)

0.8614232209737828

In [177]:
lr.score(X_test_clean, y_test)

0.8530734632683659

### Logistic Regression Pipeline

In [195]:
df_cat = X_train.select_dtypes(include=['object'])

# select numerical columns
df_num = X_train.select_dtypes(include=['int64', 'float64'])

In [187]:
num_pipe = Pipeline(steps=[('ss', StandardScaler())])


cat_pipe = Pipeline(steps=[('ohe', OneHotEncoder(drop='first', sparse=False))])

transformer = ColumnTransformer(transformers=[
    ('numerical', num_pipe, df_num.columns),
    ('categorical', cat_pipe, df_cat.columns)
])

In [192]:
model_pipe_lr = Pipeline(steps=[
    ('transformer', transformer),
    ('lr', LogisticRegression(random_state=42))
])

model_pipe_lr.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler())]),
                                                  Index(['account length', 'number vmail messages', 'total day minutes',
       'total day calls', 'total day charge', 'total eve minutes',
       'total eve calls', 'total eve charge', 'total night minutes',
       'total night calls', 'total night charge', 'total intl minutes',
       'total intl calls', 'total intl charge', 'customer service calls'],
      dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 s

In [193]:
model_pipe_lr.score(X_train, y_train)

0.872093023255814

In [194]:
model_pipe_lr.score(X_test, y_test)

0.8530734632683659

### KNN

In [134]:
knn = KNeighborsClassifier()

knn.fit(X_train_clean, y_train)

KNeighborsClassifier()

In [135]:
scores = cross_val_score(estimator=knn, X = X_train_clean, y= y_train, cv=10)
scores

array([0.87640449, 0.86516854, 0.88764045, 0.88389513, 0.88389513,
       0.89138577, 0.86842105, 0.87218045, 0.87969925, 0.90225564])

In [136]:
np.median(scores)

0.8817971896032215

In [137]:
knn.score(X_test_clean, y_test)

0.8935532233883059

In [153]:
grid = {
    'n_neighbors': [3 ,5 , 11],
    'metric': ['manhattan', 'minkowski'],
    'weights': ['uniform', 'distance']
}

In [139]:
knn

KNeighborsClassifier()

In [140]:
gs = GridSearchCV(estimator=knn, param_grid= grid, cv=5)

In [141]:
gs.fit(X_train_clean, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['manhattan', 'minkowski'],
                         'n_neighbors': [3, 5, 11],
                         'weights': ['uniform', 'distance']})

In [142]:
gs.best_params_

{'metric': 'minkowski', 'n_neighbors': 5, 'weights': 'uniform'}

This happens to be the same as our default/ original model

In [143]:
gs.best_estimator_.score(X_test_clean, y_test)

0.8935532233883059

### Pipeline for knn

In [146]:
model_pipe_knn = Pipeline(steps=[
    ('transformer', transformer),
    ('knn', KNeighborsClassifier())
])

model_pipe_knn.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler())]),
                                                  Index(['account length', 'number vmail messages', 'total day minutes',
       'total day calls', 'total day charge', 'total eve minutes',
       'total eve calls', 'total eve charge', 'total night minutes',
       'total night calls', 'total night charge', 'total intl minutes',
       'total intl calls', 'total intl charge', 'customer service calls'],
      dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 s

In [148]:
model_pipe_knn.score(X_train, y_train)

0.9036009002250562

In [149]:
model_pipe_knn.score(X_test, y_test)

0.8935532233883059

In [152]:
param_grid = {
    'knn__n_neighbors': [3, 5, 11],
    'knn__metric': ['manhattan', 'minkowski'],
    'knn__weights': ['uniform', 'distance']
}

gs = GridSearchCV(model_pipe_knn, param_grid=param_grid, cv=5)
gs.fit(X_train, y_train)
grid_search.best_params_

{'knn__metric': 'manhattan', 'knn__n_neighbors': 3, 'knn__weights': 'uniform'}

In [151]:
gs.best_estimator_.score(X_test, y_test)

0.8875562218890555

### Decision Trees

In [26]:
clf = DecisionTreeClassifier(random_state=42)

clf.fit(X_train_clean, y_train)

DecisionTreeClassifier(random_state=42)

In [27]:
clf.score(X_train_clean, y_train)

1.0

In [28]:
clf.score(X_test_clean, y_test)

0.9280359820089955

In [29]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

In [30]:
clf

DecisionTreeClassifier(random_state=42)

In [31]:
gs = GridSearchCV(clf, param_grid=param_grid, cv=5)

In [32]:
gs.fit(X_train_clean, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 3, 5, 7, 9],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]})

In [33]:
gs.best_params_

{'criterion': 'entropy',
 'max_depth': 7,
 'min_samples_leaf': 2,
 'min_samples_split': 10}

In [34]:
gs.best_estimator_.score(X_test_clean, y_test)

0.9385307346326837

### Pipeline for Decision Tree

In [168]:
model_pipe_tree = Pipeline(steps=[
    ('transformer', transformer),
    ('dt', DecisionTreeClassifier(random_state=42))
]) 

In [169]:
model_pipe_tree.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler())]),
                                                  Index(['account length', 'number vmail messages', 'total day minutes',
       'total day calls', 'total day charge', 'total eve minutes',
       'total eve calls', 'total eve charge', 'total night minutes',
       'total night calls', 'total night charge', 'total intl minutes',
       'total intl calls', 'total intl charge', 'customer service calls'],
      dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 s

In [170]:
model_pipe_tree.score(X_train, y_train)

1.0

In [171]:
model_pipe_tree.score(X_test, y_test)

0.9280359820089955

In [ ]:
# Define the hyperparameter grid to search over
param_grid = {
    'dt__criterion': ['gini', 'entropy'],
    'dt__max_depth': [None, 3, 5, 7, 9],
    'dt__min_samples_split': [2, 5, 10],
    'dt__min_samples_leaf': [1, 2, 4],
}

In [172]:
# Create a GridSearchCV object
grid_search = GridSearchCV(
    estimator=model_pipe_tree,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1
)

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and corresponding score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Best parameters:  {'dt__criterion': 'entropy', 'dt__max_depth': 7, 'dt__min_samples_leaf': 2, 'dt__min_samples_split': 10}
Best score:  0.9403594943468881


### Voting Classifier

In [202]:
w_avg = VotingClassifier(estimators=[
    ('lr', LogisticRegression()), 
    ('knn', KNeighborsClassifier(metric= 'minkowski', 
                                 n_neighbors= 5, 
                                 weights= 'uniform')),
    ('dt', DecisionTreeClassifier(criterion= 'entropy',
                                  max_depth= 7,
                                  min_samples_leaf= 2,
                                  min_samples_split= 10))
])

w_avg.fit(X_train_clean, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('knn', KNeighborsClassifier()),
                             ('dt',
                              DecisionTreeClassifier(criterion='entropy',
                                                     max_depth=7,
                                                     min_samples_leaf=2,
                                                     min_samples_split=10))])

In [203]:
scores = cross_val_score(estimator=w_avg, X=X_train_clean, y=y_train, cv=5)
scores

array([0.89513109, 0.89868668, 0.90243902, 0.87617261, 0.9043152 ])

In [204]:
np.median(scores)

0.8986866791744841

In [206]:
w_avg.score(X_train_clean, y_train)

0.9208552138034508

In [207]:
w_avg.score(X_test_clean, y_test)

0.9070464767616192

Now let's try again with re-weighting the same thing

In [210]:
w_avg = VotingClassifier(estimators=[
    ('lr', LogisticRegression()), 
    ('knn', KNeighborsClassifier(metric= 'minkowski', 
                                 n_neighbors= 5, 
                                 weights= 'uniform')),
    ('dt', DecisionTreeClassifier(criterion= 'entropy',
                                  max_depth= 7,
                                  min_samples_leaf= 2,
                                  min_samples_split= 10))],
    weights=[.2, .3, .5]
)

w_avg.fit(X_train_clean, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('knn', KNeighborsClassifier()),
                             ('dt',
                              DecisionTreeClassifier(criterion='entropy',
                                                     max_depth=7,
                                                     min_samples_leaf=2,
                                                     min_samples_split=10))],
                 weights=[0.2, 0.3, 0.5])

In [211]:
w_avg.score(X_train_clean, y_train)

0.9204801200300075

In [212]:
w_avg.score(X_test_clean, y_test)

0.9055472263868066

So far our best model has been DecisionTreeClassifier with the following parameters:
- Criterion: 'entropy'
- max_depth: 7
- min_samples_leaf: 2
- min_samples_split: 10

With an accuracy of .94 on the testing set

### Bagging

In [214]:
bag = BaggingClassifier(n_estimators=100, verbose=1, random_state=42)

In [215]:
bag.fit(X_train_clean, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s finished


BaggingClassifier(n_estimators=100, random_state=42, verbose=1)

In [216]:
scores = cross_val_score(estimator=bag, X=X_train_clean, y=y_train, cv=5)
scores

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

array([0.94569288, 0.94559099, 0.93621013, 0.94183865, 0.9587242 ])

In [218]:
np.median(scores)

0.9455909943714822

In [220]:
bag.score(X_test_clean, y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


0.9535232383808095

In [246]:
y_pred = bag.predict(X_test_clean)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.96      0.99      0.97       566
        True       0.93      0.75      0.83       101

    accuracy                           0.95       667
   macro avg       0.94      0.87      0.90       667
weighted avg       0.95      0.95      0.95       667



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


So now we have a model with 95% accuracy, our best model yet! However, our recall is a meager 75%. Let's see if we can improve that by tweaking the threshold

In [254]:
y_prob = bag.predict_proba(X_test_clean)[:,1]
y_pred = (y_prob >= 0.32).astype(int)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.97      0.96      0.97       566
        True       0.80      0.83      0.82       101

    accuracy                           0.94       667
   macro avg       0.88      0.90      0.89       667
weighted avg       0.94      0.94      0.94       667



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [255]:
print(confusion_matrix(y_test, y_pred))

[[545  21]
 [ 17  84]]


The best we can do without changing the model too much is a recall of .83 while only decreasing the accuracy by 1%. However, the precision took a big hit and went down over 10%. As mentioned earlier this is not such a big deal. Although, it may still not be worth the trade off.